In [1]:
import os


In [2]:
%pwd

'd:\\ML\\spelling correction\\Spelling-correction-project\\resreach'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML\\spelling correction\\Spelling-correction-project'

In [5]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from SpellingCorrection.constants import*
from SpellingCorrection.utils.common import read_yaml,create_directories

In [7]:
import os
import yaml
from dataclasses import dataclass 
from pathlib import Path
CONFIG_FILE_PATH = Path("config/config.yaml")  # Replace with the actual file path
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from SpellingCorrection.logging import logger
from SpellingCorrection.utils.common import get_size


In [9]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                                                url=self.config.source_URL,
                                                filename=self.config.local_data_file
                                                )
            logger.info(f"{filename} downloaded! with following headers: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """""
        zip_file_path: str
        Extracts the zip file to the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e



[2024-06-10 14:18:50,581 : INFO : common : Reading the yaml file from the path: config\config.yaml loaded successfully]
[2024-06-10 14:18:50,586 : INFO : common : Reading the yaml file from the path: params.yaml loaded successfully]
[2024-06-10 14:18:50,588 : INFO : common : Creating the directory: artifacts]
[2024-06-10 14:18:50,589 : INFO : common : Creating the directory: artifacts/data_ingestion]
[2024-06-10 14:19:32,025 : INFO : 3822425393 : artifacts/data_ingestion/dataset.zip downloaded! with following headers: 
Connection: close
Content-Length: 16211735
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "b10d1bbc215d724995fc0b6ce42d46021b641d75c3dda7d8cd58c12ab9b3d10d"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5004:4FE90:3E53:4942:6666A8DF
Accept-Ranges: bytes
Date: Mon, 10 Jun 20